# TAPL Subtyping

Presented by [Pinglei Guo](https://github.com/at15) for [UCSC CMPS253](https://courses.soe.ucsc.edu/courses/cmps253) on 11/07/2017

## Table of content

NOTE: the title and order of some sections are modfied to emphasize specific content

- [Notation](#notation)
- [Chapter 15 Subtyping](#subtyping)
  - [15.1 Motivation](#motivation)
  - [15.2 Record](#record)
  - [15.5 Variants](#variants)
  - [15.2 **S-Arrow**](#s-arrow)
  - 15.3 Progress & Preservation (skipped)
  - [15.4 Top & Bottom](#top-bottom)
  - [15.5 Cast, Reference](#cast-reference)
  - 15.6 Coercion Semantics for Subtyping (skipped)
- [Chapter 16 Metatheory of Subtyping](#metatheory)
  - [16.1 Algorithmic Subtyping](#algorithmic-subtyping)
  - [16.2 Algorithmic Typing](#algorithmic-typing)
- Chapter 17 An ML Implementation of Subtyping
- [Take away](#take-away)
- [Reference](#reference)

## Notation

We use a more Java-style pseudo code instead of lambda-calculus to have named function and multi arguments

The style we should use

````ocaml
(* NOTE: from [2], rewrite in a mix of OCaml and lambda-calculus, won't compile *)
let sphere = {center={x=1, y=2, z=3}, radius=2} in
    let circle = sphere in 
        (lambda c:{center: {x: Nat, y:Nat}, radius: Nat}. c.center={x=0, y=0}) circle
    sphere.center.z
````

The style we actually use (similar to [2])

````javascript
/* {center: {x:Nat, y:Nat}, radius: Nat} -> Unit */
function move_to_origin(c: {center: {x:Nat, y:Nat}, radius: Nat}) : Unit {
    c.center = {x=0, y=0}
}
var sphere = {center={x=1, y=2, z=3}, radius=2}
move_to_origin(sphere)
sphere.center.z
````

## 15 Subtyping <a name="subtyping"></a>

### 15.1 Motivation <a name="motivation"></a>

Recall the typing rule for function application

$$\frac{\Gamma \vdash t_1 : T_{11} \rightarrow T_{12} \quad \Gamma \vdash t_2 : T_{11}}{\Gamma \vdash t_1 \ t_2 : T_{12}} \quad\quad \textbf{(T-APP)}$$

This is a well typed term

$$(\lambda \text{r:{x:Nat}. r.x) {x=0}}$$

This is not typable, but it looks reasonable

$$(\lambda \text{r:{x:Nat}. r.x) {x=0,y=1}}$$

However, passing {x=0,y=1} to ($\lambda$ r:{x:Nat}. r.x) should yield 0, the function only needs field x and don't care other fields in record. So we extend our rule to allow this kind of operation, where a more informative type ({x:Nat, y:Nat}) is used in the place of a less informative one ({x:Nat}).

We can use {x=0, y=1} in palace of {x=0}, we say {x:Nat, y:Nat} is a subtype, written <: of {x:Nat}. It's sub because the number of records of the subtype is less. [3]

Foramally, S is a *subtype* of T written S <: T, mean any term of type S can safely be used in a context where a term of type T is expected

$$\frac{\Gamma \vdash t : S \quad S <: T}{\Gamma \vdash t : T} \quad\quad \textbf{(T-SUB)}$$

Intuitively, our *subtype* relation also have reflexivity and transitivity

$$ S <: S \quad \textbf{(S-REFL)}$$

$$\frac{S <:U \quad U <: T}{S <: T} \quad\quad \textbf{(S-TRANS)}$$

**Question**

Anything strange about our T-SUB rule compared w/ other typing rules when syntax is considered?

Syntax

````
t :: = x               variable
     lambda x:T. t     abstraction (function)
     t t               application (function call)
````

Typing rules

$$\frac{\Gamma,x:T_1 \vdash t_2:T_2}{\Gamma \vdash \lambda x:T_1. t_2:T_1 \rightarrow T_2} \quad\quad \textbf{(T-ABS)}$$


$$\frac{\Gamma \vdash t_1 : T_{11} \rightarrow T_{12} \quad \Gamma \vdash t_2 : T_{11}}{\Gamma \vdash t_1 \ t_2 : T_{12}} \quad\quad \textbf{(T-APP)}$$


$$\frac{\Gamma \vdash t : S \quad S <: T}{\Gamma \vdash t : T} \quad\quad \textbf{(T-SUB)}$$

This will be answered in [Chapter 16](#metatheory)

### 15.2 Record <a name="record"></a>

We formalize {x=0, y=1} <: {x=0} as **width subtyping** rule

$$\{l_i:T_i^{i \in 1..n+k}\} <: \{l_i:T_i^{i \in 1..n}\} \quad \textbf{(S-RCDWIDTH)}$$

It might looks counter intuitive that the "smaller" type (subtype) actually has more fields. That is because more fields in the record type means more restriction and a smaller set it can cover, the following [euler diargam](https://en.wikipedia.org/wiki/Euler_diagram) shows this relation, {x:Nat, y:Nat, z:Nat} <: {x:Nat, y:Nat} <: {x:Nat} and {x:Nat, y:Nat} <: {y:Nat}



The "smaller" record type has more fields because there are less records of this type

![record-width](record-width.png)

**Question**

If we specialize record to tuple, how many circle would left in above diagram?

In **S-RcdWidth**, we require each field has identitical type, however, this restriction can be loosen, they can have subtype relation, this gives us the **depth subtyping** rule

$$\frac{\text{for each } i \quad S_i <: T_i}{\{l_i:S_i^{i \in 1..n}\} <: \{l_i:T_i^{i \in 1..n}\}} \quad\quad \textbf{(S-RCDDEPTH)}$$

We can combine **S-RcdWidth** and **S-RcdDepth** together

$$\frac{
\text{\{ x: Nat, y: Nat, z: Nat\}} <: \text{\{x: Nat, y: Nat\}} \quad 
\text{\{radius: Nat\}} <: \text{\{radius: Nat\}}
}
{\text{\{center: {x: Nat, y: Nat, z: Nat}, radius: Nat\}} <: \text{\{center: {x: Nat, y: Nat}, radius: Nat\}} }$$

**Question**

If we allow mutable record, what would happen?

````javascript
/* {center: {x:Nat, y:Nat}, radius: Nat} -> Unit */
function move_to_origin(c: {center: {x:Nat, y:Nat}, radius: Nat}) : Unit {
    c.center = {x=0, y=0}
}
var sphere = {center={x=1, y=2, z=3}, radius=2}
move_to_origin(sphere)
sphere.center.z // 3?
````

This will be answered in [15.5](#cast-reference)

The order of fields in record type does not have impact on safety because our projection is insensitive to it, so we have the last rule for record **S-RcdPerm**

$$\{l_i:T_i^{i \in 1..n+k}\} <: \{l_i:T_i^{i \in 1..n}\} \quad \textbf{(S-RCDWIDTH)}$$

$$\frac{\text{for each } i \quad S_i <: T_i}{\{l_i:S_i^{i \in 1..n}\} <: \{l_i:T_i^{i \in 1..n}\}} \quad\quad \textbf{(S-RCDDEPTH)}$$

$$\frac{\{k_j:S_j^{j\in 1..n}\} \text{ is a permuatation of } \{l_i:T_i^{i\in 1..n}\}}
{\{k_j:S_j^{j\in 1..n}\} <: \{l_i:T_i^{i\in 1..n}\}} 
\quad\quad \textbf{(S-RCDPERM)}$$

In fact, record subtyping is not widely adopted, OCaml is using row-variable polymorphism for records [1].

### 15.5 Variants <a name="variants"></a>

Variants is similar to record in some sense, record is a combination of many types, variant is also a combination of many types, but it can only be one of them at given time. Intuitively we have depth and permutation rule for variants.

$$\frac{\text{for each } i \quad S_i <: T_i}{\{l_i:S_i^{i \in 1..n}\} <: \{l_i:T_i^{i \in 1..n}\}} \quad\quad \textbf{(S-RCDDEPTH)}$$

$$\frac{\text{for each } i \quad S_i <: T_i}{\langle l_i:S_i^{i \in 1..n}\rangle <: \langle l_i:T_i^{i \in 1..n}\rangle} \quad\quad \textbf{(S-VARIANTDEPTH)}$$


$$\frac{\{k_j:S_j^{j\in 1..n}\} \text{ is a permuatation of } \{l_i:T_i^{i\in 1..n}\}}
{\{k_j:S_j^{j\in 1..n}\} <: \{l_i:T_i^{i\in 1..n}\}} 
\quad\quad \textbf{(S-RCDPERM)}$$

$$\frac{\langle k_j:S_j^{j\in 1..n}\rangle \text{ is a permuatation of } \langle l_i:T_i^{i\in 1..n}\rangle}
{\langle k_j:S_j^{j\in 1..n}\rangle <: \langle l_i:T_i^{i\in 1..n}\rangle}
\quad\quad \textbf{(S-VARIANTPERM)}$$


But what about width rule?

$$\{l_i:T_i^{i \in 1..n+k}\} <: \{l_i:T_i^{i \in 1..n}\} \quad \textbf{(S-RCDWIDTH)}$$

$$\langle l_i:T_i^{i \in 1..n}\rangle <: \langle l_i:T_i^{i \in 1..n+k}\rangle \quad \textbf{(S-VARIANTWIDTH)}$$

Why the opposite in record and variants?

- record: more fields means more requirement, less choice, thus subtype
- variants: more fields means more choice, thus supertype

![record-variant](record-variant-width.png)


### 15.2 S-Arrow <a name="s-arrow"></a>

In a high order language, function can be argument to other function. Recall our subtyping rule, t does not have to be a variable term.

$$\frac{\Gamma \vdash t : S \quad S <: T}{\Gamma \vdash t : T} \quad\quad \textbf{(T-SUB)}$$

The text book says we should use this one for function type

$$\frac{T_1 <: S_1 \quad S_2 <: T_2}{S_1 \rightarrow S_2 <: T_1 \rightarrow T_2} \quad\quad (\textbf{S-ARROW})$$

But there are total 4 combination, what about the other 3?

$$\frac{S_1 <: T_1 \quad S_2 <: T_2}{S_1 \rightarrow S_2 <: T_1 \rightarrow T_2} \quad\quad (\textbf{S-ARROW-1})$$

$$\frac{T_1 <: S_1 \quad T_2 <: S_2}{S_1 \rightarrow S_2 <: T_1 \rightarrow T_2} \quad\quad (\textbf{S-ARROW-2})$$

$$\frac{S_1 <: T_1 \quad T_2 <: S_2}{S_1 \rightarrow S_2 <: T_1 \rightarrow T_2} \quad\quad (\textbf{S-ARROW-4})$$

Note on the left side of $\rightarrow$, the relation is *contravariant* while on the right side, the relation is *covariant*, which is pretty counter intuitive. One might expect the left side (arugment) is convariant. Possible source of confusion are:

- in previous example of function application, we replace argument with its subtype. i.e. replace {x:Nat} with {x:Nat, y:Nat}
- only think about the argument of the function, NOT taking function as a whole. [5]

Let's look at a concrete example before we dive into this rule. 

Assume we have a game called Game of the Type, you are the main character John Calculus, you want to travel to the north safely, you can use the following transportation for 10h before walk all the rest

- a dragon rent from Nights King, can fly or crawl on the ground if you like
- a train called ruby, can only move veritically on rails

Following is the Pseudo-Code, the implementation is written in python and use dict to represent record.

````js
/* 
* Game of the Type - The Journey of John Calculus
*/
var start = {x:0, y:0} // your home
var end = {x:101, y:505} // the wall

function walk(p:{x:Nat, y:Nat}) : {x:Nat, y:Nat} {
    return {x=p.x+1, y=p.y+5}
}

function train(p:{y:Nat}) : {y:Nat} {
    return {y=p.y+50}
}

function dragon(p:{x:Nat, y:Nat, z:Nat}) : {x:Nat, y:Nat, z:Nat} {
    return {x=p.x+10, y=p.y+50, z=p.z+1}
}

function travel(transport:{x:Nat, y:Nat} -> {x:Nat, y:Nat}, s:{x:Nat, y:Nat}, e:{x:Nat, y:Nat}) : Unit {
    p := s
    for i := 0; i < 10; i++ {
        p := transport(p)
    }
    while (p < e) {
        p := walk(p)
    }
}
````

Case-0, walk all the way (exact same type)

$$S_1 = T_1 = \text{\{x:Nat, y:Nat\}}=  S_2 = T_2$$

In [6]:
start = {'x': 0, 'y': 0}
end = {'x': 101, 'y': 505}

# {x:Nat, y:Nat} -> {x:Nat, y:Nat}
def walk(p):
    return {'x': p['x']+1, 'y': p['y']+5}

# ({x:Nat, y:Nat} -> {x:Nat, y:Nat}) -> {x:Nat, y:Nat} -> {x:Nat, y:Nat} -> Unit
def travel(transport, s, e):
    p = s
    for i in range(10):
        p = transport(p)
    j = 0
    while (p['y'] < e['y']):
        p = walk(p)
        j += 1
    print('{} for 10h and walk {}h'.format(transport.__name__, j))
    
travel(walk, start, end)    

walk for 10h and walk 91h


Case-1, dragon, covariant for both argument and return value

$$S_1 = S_2 = \text{\{x:Nat, y:Nat, z:Nat\}} <: \text{\{x:Nat, y:Nat}\} = T_1 = T_2$$

$$\frac{S_1 <: T_1 \quad S_2 <: T_2}{S_1 \rightarrow S_2 <: T_1 \rightarrow T_2} \quad\quad (\textbf{S-ARROW-1})$$

In [7]:
# {x:Nat, y:Nat, z:Nat} -> {x:Nat, y:Nat, z:Nat}
def dragon(p):
    return {'x': p['x']+10, 'y': p['y']+50, 'z': p['z']+1}

In [8]:
travel(dragon, start, end)

KeyError: 'z'

In fact, covariant for both argument and return value sometimes does work

In [13]:
travel(dragon, {'x': start['x'], 'y': start['y'], 'z': 0}, {'x': end['x'], 'y': end['y'], 'z': 0})

dragon for 10h and walk 1h


But our type system need to be sound, not might work when special input is provided

Case-2, train, contravariant for both argument and return value

$$T_1 = T_2 = \text{\{x:Nat, y:Nat\}} <: \text{\{y:Nat}\} = T_1 = T_2$$

$$\frac{T_1 <: S_1 \quad T_2 <: S_2}{S_1 \rightarrow S_2 <: T_1 \rightarrow T_2} \quad\quad (\textbf{S-ARROW-2})$$

In [9]:
# {y:Nat} -> {y:Nat}
def train(p):
    return {'y': p['y']+50}

In [10]:
travel(train, start, end)

KeyError: 'x'

Case-3, train_dragon, contravariant for argument and covariant for return value

$$ \text{\{x:Nat, y:Nat, z:Nat\}} = S_2 <: T_2 = \text{\{x:Nat, y:Nat\}} = T_1 <: S_1 = \text{\{y:Nat\}}$$
$$\frac{T_1 <: S_1 \quad S_2 <: T_2}{S_1 \rightarrow S_2 <: T_1 \rightarrow T_2} \quad\quad (\textbf{S-ARROW})$$

In [16]:
# {y:Nat} -> {x:Nat, y:Nat, z:Nat}
def train_dragon(p):
    x = p['x']
    p = train(p)
    p = {'x': x, 'y': p['y'], 'z': 0} # get off the train and get on the dragon
    return dragon(p)

In [12]:
travel(train_dragon, start, end)

train_dragon for 10h and walk 0h


Case-4, dragon_train, covariant for argument and contravariant for return value

$$ \text{\{x:Nat, y:Nat, z:Nat\}} = S_1 <: T_1 = \text{\{x:Nat, y:Nat\}} = T_2 <: S_2 = \text{\{y:Nat\}}$$
$$\frac{S_1 <: T_1 \quad T_2 <: S_2}{S_1 \rightarrow S_2 <: T_1 \rightarrow T_2} \quad\quad (\textbf{S-ARROW-4})$$

In [17]:
# {x:Nat, y:Nat, z:Nat} -> {y:Nat}
def dragon_train(p):
    p = {'x': p['x'], 'y': p['y'], 'z': 0}
    return train(dragon(p))

In [15]:
travel(dragon_train, start, end)

KeyError: 'x'

Another way to think about is treat function as funnel [5] TODO: illustration

### 15.4 Top & Bottom <a name="top-bottom"></a>

### 15.5 Cast, Reference <a name="cast-reference"></a>

## 16 Metatheory of Subtyping

### 16.1 Algorithmic Subtyping

### 16.2 Algorithmic Typing

## Take away <a name="take-away"></a>

## Reference <a name="reference"></a>

Books

- [1] Types and Programming Language (2nd Edition)

Courses

- [2] [Coursera Programming Languages Part C Week 3](https://www.coursera.org/learn/programming-languages-part-c/lecture/mdwdY/subtyping-from-the-beginning)
- [3] [EPFL Type Systems 2004](http://lampwww.epfl.ch/teaching/archive/type_systems/2004/)
- [4] [EPFL Foundations of Software 2017](https://fos2017.github.io/)

Meetup

- [5] [Computational Club (UK) TAPL reading meetup](https://github.com/computationclub/computationclub.github.io/wiki/Types-and-Programming-Languages-Chapter-15-Subtyping-%E2%80%93-Part-1)

Miscellaneous

- http://www.slideviper.oquanta.info/tutorial/slideshow_tutorial_slides.html#/
- [Presenting slide using Jupyter notebook](https://medium.com/@mjspeck/presenting-code-using-jupyter-notebook-slides-a8a3c3b59d67)